In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Читаем данные

In [90]:
data = pd.read_excel('ecology_cleaned 1.xlsx')
print('Форма данных:', data.shape)

Форма данных: (1323, 36)


## Пропущенные значения в колонках

In [91]:
data.isnull().sum()[data.isnull().sum() > 0]

Как часто фирма отслеживала потребление энергии последние 3 года?                              542
Как часто фирма отслеживала потребление воды последние 3 года?                                 852
За последние 3 года фирма отслеживала выбросы CO2 (углекислого газа)?                         1157
Какая мера в наибольшей степени способствовала уменьшению воздействия на окружающую среду?     652
Какова основная причина того, что меры для повышения энергоэффективности не применялись?       447
dtype: int64

# Создание индекса вовлеченности

In [92]:
eco_factors = [
    "Улучшение систем подогрева и охлаждения",
    "Применение более благоприятных для климата источников энергии",
    "Модернизация оборудования",
    "Управление энергопотреблением",
    "Минимизация отходов, переработка",
    "Меры по ограничению загрязнения воздуха",
    "Управление потреблением воды",
    "Модернизация транспортных средств, судов, самолетов",
    "Усовершенствование системы освещения",
    "Меры по ограничению загрязнения окружающей среды",
    "Последние 3 года фирма принимала меры для повышения энергоэффективности?",
    "Последние 3 года фирма имела целевые уровни по потреблению энергии?",
    "За последние 3 года фирма осуществляла выбросы CO2 (углекислого газа)?"
]

print('Количество признаков на вовлеченность в экологию: ', len(eco_factors))

Количество признаков на вовлеченность в экологию:  13


In [93]:
def replace_values(column):
    return column.map({'Да': 1, 'Нет': 0}).fillna(-1).astype(int)

cols = []
for i in data.columns:
    cols.append(i.strip().lower())

for column_name in eco_factors:
    col = column_name.strip().lower()
    if col in cols:
        data[column_name] = replace_values(data[column_name])

In [94]:
data['involve in ecology index'] = -1

for i in range(len(data)):
    result_index = 0 # итоговый индекс вовлеченности компании

    industry = data['Отрасль экономики, к которой относится фирма'][i]
    n_companies_in_industry = len(data[data['Отрасль экономики, к которой относится фирма'] == industry])

    summary = 0
    sum_weights = 0

    for factor in eco_factors:
        n_yes_companies_in_industry = len(data[(data['Отрасль экономики, к которой относится фирма'] == industry) & (data[factor] == 1)])
        if n_yes_companies_in_industry != 0:
            weight = float(n_companies_in_industry) / n_yes_companies_in_industry
        else:
            continue
        
        if data[factor][i] != -1:
            summary += float(data[factor][i]) * weight
            sum_weights += weight

    if sum_weights == 0:
        result_index = 0
    else:
        result_index = summary / sum_weights

    data['involve in ecology index'][i] = result_index

/var/folders/mh/lrx97nzx0f785c2syyjkkc3w0000gn/T/ipykernel_2207/2813948732.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['involve in ecology index'][i] = result_index
/var/folders/mh/lrx97nzx0f785c2syyjkkc3w0000gn/T/ipykernel_2207/2813948732.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.25462023674021994' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data['involve in ecology index'][i] = result_index


In [ ]:
data.to_excel('ecology_with_index.xlsx')